# This is the file illustrating simulation

In [ ]:
## modeules to use
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import linear_model
from sklearn import model_selection
from sklearn.svm import SVC,SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
import sklearn.metrics as metrics
from sklearn.ensemble import StackingRegressor, StackingClassifier
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
from scipy import linalg, special
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statistics as stat
import seaborn as sns
import time
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import multiprocess
import statsmodels.formula.api as smf
from scipy.stats import logistic
import os 
from scipy.linalg import toeplitz


In [ ]:
# simulate data
def simulation_base(p,n,rho,beta_gn,sz,sz0):
    # complex relationship with fixed true model
    from itertools import combinations
    p0 = 20; prest = p-p0
    interind = np.array([np.array(list(x)) for x in combinations(range(2, len(beta_gn), len(beta_gn)//5), 2)])

    z00 = np.random.binomial(n=1, p=0.35, size=n)

    # correlated continuous covariates
    sig = toeplitz([rho ** i for i in range(p0-1)])
    z0 = np.random.multivariate_normal(mean=np.zeros(p0-1), cov=sig, size=n)
    Z0 = np.concatenate([z00[:, None], z0], axis=1)
    interterm = np.apply_along_axis(lambda x: Z0[:, x[0]] * Z0[:, x[1]], axis=1, arr=interind)
    costerm = np.apply_along_axis(lambda x: np.cos(Z0[:,x]),axis = 0, arr = np.unique(interind))
    puse0 = 1/(1+(np.exp(-0.2*np.dot(Z0, beta_gn)- 0.1 * interterm.sum(axis=0))))
    t0 = np.random.binomial(n=1, p=puse0, size=n)

    y0_1 = np.random.normal(loc=sz0 + np.dot(Z0, beta_gn)+interterm.sum(axis=0), scale=1)
    y0_0 = np.random.normal(loc=np.dot(Z0, beta_gn)+interterm.sum(axis=0)+costerm.sum(axis = 1), scale=1)
    y0 = np.where(t0==1, y0_1, y0_0)

    Zaux0 = np.random.normal(0,1,size = (n,prest))

    dat0 = pd.DataFrame(np.concatenate([Z0, Zaux0, t0[:, None], y0[:, None]], axis=1), columns=["z00"] + [f"z{i}0" for i in range(1, p)] + ["t0", "y0"])

    # time point 2
    z01 = z00 + np.random.uniform(size=n)

    z1_1 = z0 + 0.1*y0_1[:, np.newaxis]
    z1_0 = z0 + 0.1*y0_0[:, np.newaxis]
    z1 = np.concatenate([z1_1[t0==1, :], z1_0[t0==0, :]], axis=0)

    Z1_1 = np.concatenate([z01[:, None], z1_1], axis=1)
    Z1_0 = np.concatenate([z01[:, None], z1_0], axis=1)
    Z1 = np.concatenate([z01[:, None], z1], axis=1)

    interterm1 = np.apply_along_axis(lambda x: Z1[:, x[0]] * Z1[:, x[1]], axis=1, arr=interind)
    costerm1 = np.apply_along_axis(lambda x:np.cos(Z1[:,x]),axis = 0, arr = np.unique(interind))
    puse1 = 1/(1+np.exp(-0.1*np.dot(Z1, beta_gn)+0.5-0.05*interterm1[range(5,interterm1.shape[0]),:].sum(axis = 0)))
    t1 = np.random.binomial(n=1, p=puse1, size=n)

    y1_11 = np.random.normal(loc=sz + np.dot(Z1_1, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1_00 = np.random.normal(loc=np.dot(Z1_0, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0), scale=1)
    y1_10 = np.random.normal(loc=np.dot(Z1_1, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1_01 = np.random.normal(loc=sz + np.dot(Z1_0, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1 = np.where(t0==1, np.where(t1==1, y1_11, y1_10), np.where(t1==0, y1_00, y1_01))

    Zaux1 = np.random.normal(0,1,size = (n,prest))

    dat1 = pd.DataFrame(np.concatenate([Z1, Zaux1, t1[:, None], y1[:, None]], axis=1), columns=["z01"] + [f"z{i}1" for i in range(1, p)] + ["t1", "y1"])
    
    full_data = pd.concat([dat0, dat1], axis=1)
    return full_data, pd.DataFrame({'y_11': y1_11, 'y_00': y1_00, 'p0': puse0, 'p1':puse1})



In [ ]:
# PS function
def ps_train_ensemble(data,Train,Test,features,outcomes,
                      include_lg=True, include_el=True, include_xgb=True, include_mlp=False,
                      include_l1=False, include_svm=False, include_nb=False,include_rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lg:
        lg = linear_model.LogisticRegression()
        lg.fit(data[features],data[outcomes])
        models['logit'] = lg

    # base learner 2
    if include_el:
        el_class = linear_model.SGDClassifier(penalty='elasticnet',loss='log')
        param_grid_el = {'alpha': [0.0001, 0.001, 0.01, 0.1]}
        gs_el = GridSearchCV(estimator=el_class,param_grid=param_grid_el,cv=5,scoring='neg_log_loss')
        gs_el.fit(Train[features],Train[outcomes])
        el_class.set_params(**gs_el.best_params_)
        el_class.fit(Train[features],Train[outcomes])
        models['enet'] = el_class

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
         'max_depth': [55,65,80],
        # 'n_estimators': [100,200,300],
        # 'alpha':[0.05,0.1,0.18],
        # 'gamma': [1,1.5,1.8]
        }
        xgb_cls = xgb.XGBClassifier(objective='binary:logistic')
        gs_xgb = GridSearchCV(
            estimator=xgb_cls,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_log_loss'
        )
        gs_xgb.fit(Train[features], Train[outcomes])
        xgb_cls.set_params(**gs_xgb.best_params_)
        xgb_cls.fit(Train[features], Train[outcomes])
        models['xgb'] = xgb_cls

    # base learner 4
    if include_mlp:
        mlp_cls = MLPClassifier()
        param_grid_mlpc = {
            'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)],
        }
        gs_mlp = GridSearchCV(estimator=mlp_cls,param_grid=param_grid_mlpc,cv=5,scoring="neg_log_loss")
        gs_mlp.fit(Train[features],Train[outcomes])
        mlp_cls.set_params(**gs_mlp.best_params_)
        mlp_cls.fit(Train[features],Train[outcomes])
        models['mlp'] = mlp_cls

    if include_l1:
        l1_cls = linear_model.LogisticRegression(penalty='l1',solver='saga')
        param_grid_l1 = {
        'C':[0.1,1,5],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_l1 = GridSearchCV(estimator=l1_cls,param_grid=param_grid_l1,cv=5,scoring="neg_log_loss")
        gs_l1.fit(Train[features],Train[outcomes])
        l1_cls.set_params(**gs_l1.best_params_)
        l1_cls.fit(Train[features],Train[outcomes])
        models['gbr'] = l1_cls

    if include_svm:
        svm_c = SVC(kernel='linear',probability=True)
        gs_svm = GridSearchCV(estimator=svm_c,param_grid={'C': [0.1,1,10,100]},cv=5,scoring='neg_log_loss')
        gs_svm.fit(Train[features],Train[outcomes])
        svm_c.set_params(**gs_svm.best_params_)
        svm_c.fit(Train[features],Train[outcomes])
        models['svm'] = svm_c
        
    if include_nb:
        nb_c = GaussianNB()
        param_grid_nb = {
            'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
            }
        gs_nb = GridSearchCV(nb_c, param_grid=param_grid_nb, cv=5)
        gs_nb.fit(Train[features],Train[outcomes])
        nb_c.set_params(**gs_nb.best_params_)
        nb_c.fit(Train[features],Train[outcomes])
        models['nb'] = nb_c

    if include_rf:
        rf_c = RandomForestClassifier(random_state = 42)
        param_grid_rf = {
            'n_estimators':[100,200],
            'max_depth':[40,50],
            'min_samples_split':[2,5],
            }
        gs_rf = GridSearchCV(rf_c, param_grid=param_grid_rf, cv=5, scoring='neg_log_loss')
        gs_rf.fit(Train[features],Train[outcomes])
        rf_c.set_params(**gs_rf.best_params_)
        rf_c.fit(Train[features],Train[outcomes])
        models['rf'] = rf_c

    # meta learner
    Q1fit = []
    if include_lg:
        Q1fit.append(lg.predict_proba(Test[features])[:,1].ravel())
    if include_el:
        Q1fit.append(el_class.predict_proba(Test[features])[:,1].ravel())
    if include_xgb:
        Q1fit.append(xgb_cls.predict_proba(Test[features])[:,1].ravel())
    if include_mlp:
        Q1fit.append(mlp_cls.predict_proba(Test[features])[:,1].ravel())
    if include_l1:
        Q1fit.append(l1_cls.predict_proba(Test[features])[:,1].ravel())
    if include_svm:
        Q1fit.append(svm_c.predict_proba(Test[features])[:,1].ravel())
    if include_nb:
        Q1fit.append(nb_c.predict_proba(Test[features])[:,1].ravel())
    if include_rf:
        Q1fit.append(rf_c.predict_proba(Test[features])[:,1].ravel())

    cls_stack = linear_model.LogisticRegression()
    cls_stack.fit(np.transpose(Q1fit),Test[outcomes])

    return models, cls_stack

def ps_function(data,features,outcomes, lg=True, el=True, xgb=True, mlp=False,
                l1=False, svm=False, nb=False,rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = ps_train_ensemble(data,Train,Test,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    model_ts,stack_ts = ps_train_ensemble(data,Test,Train,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    return model_tr,model_ts,stack_tr,stack_ts

def ps_predict(preddat,features,model_tr,model_ts,stack_tr,stack_ts):
    y_pred_tr = {}
    for name, model in model_tr.items():
        y_pred_tr[name] = model.predict_proba(preddat[features])[:,1].ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack_tr.predict_proba(Q_base_tr)[:,1]

    y_pred_ts = {}
    for name, model in model_ts.items():
        y_pred_ts[name] = model.predict_proba(preddat[features])[:,1].ravel()
    Q_base_ts = np.transpose(np.array(list(y_pred_ts.values())))
    eta_ts = stack_ts.predict_proba(Q_base_ts)[:,1]

    return eta_tr,eta_ts

In [ ]:
# Gcomputation function
def gcompu_train_ensemble(data,Train,Test,features,outcomes,
                          include_lm = True, include_rg = False, 
                          include_xgb = True, include_mlp = False,
                          include_gbr = False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lm:
        lm_1 = linear_model.LinearRegression()
        lm_1.fit(data[features],data[outcomes])
        models['lm'] = lm_1

    # base learner 2
    if include_rg:
        rg_lm = linear_model.Ridge()
        param_grid_rg = {'alpha': [0.1, 1, 10]}
        gs_rg = GridSearchCV(estimator=rg_lm,param_grid=param_grid_rg,cv=5,scoring="neg_mean_absolute_error")
        gs_rg.fit(Train[features],Train[outcomes])
        rg_lm.set_params(**gs_rg.best_params_)
        rg_lm.fit(Train[features],Train[outcomes])
        models['rg'] = rg_lm

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [15,20,25,35],
        # 'n_estimators': [100, 150,200],
        #'alpha':[0.05,0.1,0.18],
        #'gamma': [1,1.5,1.8]
        }
        xgb_lm1_tr = xgb.XGBRegressor()
        gs_gcomp1_tr = GridSearchCV(
            estimator=xgb_lm1_tr,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_mean_absolute_error'
        )
        gs_gcomp1_tr.fit(Train[features], Train[outcomes])
        xgb_lm1_tr.set_params(**gs_gcomp1_tr.best_params_)
        xgb_lm1_tr.fit(Train[features], Train[outcomes])
        models['xgb'] = xgb_lm1_tr

    # base learner 4
    if include_mlp:
        mlp_lm = MLPRegressor()
        param_grid_mlpr = {
            'hidden_layer_sizes': [ (100,), (150,), (100, 100), (150,150)],
            'alpha': [0.0001, 0.001, 0.01, 0.1],
        }
        gs_mlp_lm = GridSearchCV(estimator=mlp_lm,param_grid=param_grid_mlpr,cv=5,scoring="neg_mean_absolute_error")
        gs_mlp_lm.fit(Train[features],Train[outcomes])
        mlp_lm.set_params(**gs_mlp_lm.best_params_)
        mlp_lm.fit(Train[features],Train[outcomes])
        models['mlp'] = mlp_lm

    if include_gbr:
        gbr_lm = GradientBoostingRegressor()
        param_grid_gbr = {
        # 'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [15,20,25,35],
        #'min_samples_split': [2, 3, 4],
        #'min_samples_leaf': [1, 2, 3],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_gbr_lm = GridSearchCV(estimator=gbr_lm,param_grid=param_grid_gbr,cv=5,scoring="neg_mean_absolute_error")
        gs_gbr_lm.fit(Train[features],Train[outcomes])
        gbr_lm.set_params(**gs_gbr_lm.best_params_)
        gbr_lm.fit(Train[features],Train[outcomes])
        models['gbr'] = gbr_lm

    # meta learner
    Q1fit = []
    if include_lm:
        Q1fit.append(lm_1.predict(Test[features]).ravel())
    if include_rg:
        Q1fit.append(rg_lm.predict(Test[features]).ravel())
    if include_xgb:
        Q1fit.append(xgb_lm1_tr.predict(Test[features]).ravel())
    if include_mlp:
        Q1fit.append(mlp_lm.predict(Test[features]).ravel())
    if include_gbr:
        Q1fit.append(gbr_lm.predict(Test[features]).ravel())

    lm_stack = linear_model.LinearRegression()
    lm_stack.fit(np.transpose(Q1fit),Test[outcomes])

    return models, lm_stack

def gcompu_function(data,features,outcomes,lm = True, rg = False, 
                    xgb = True, mlp = False,gbr = False):
    import warnings
    warnings.filterwarnings('ignore')
    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = gcompu_train_ensemble(data,Train,Test,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    model_ts,stack_ts = gcompu_train_ensemble(data,Test,Train,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    return model_tr,model_ts,stack_tr,stack_ts


def gcompu_predict(preddat,features,model_tr,model_ts,stack_tr,stack_ts):
    y_pred_tr = {}
    for name, model in model_tr.items():
        y_pred_tr[name] = model.predict(preddat[features]).ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack_tr.predict(Q_base_tr)
    y_pred_ts = {}
    for name, model in model_ts.items():
        y_pred_ts[name] = model.predict(preddat[features]).ravel()
    Q_base_ts = np.transpose(np.array(list(y_pred_ts.values())))
    eta_ts = stack_ts.predict(Q_base_ts)

    return eta_tr,eta_ts

In [ ]:
filename = 'yourfilename'
np.random.seed(123)
p0 = 20; 
beta_gn = np.zeros(p0)
selected_indices = np.random.choice(np.arange(p0), size=p0-10, replace=False)
beta_gn[selected_indices] = np.random.normal(size=p0-10)

In [ ]:
def simulation_loop(b,n = 1000,p = 50,filename = filename,beta_gn = beta_gn):
    import warnings
    warnings.filterwarnings('ignore')
    print(p)
    np.random.seed(123+b)
    testdat, paramdat = simulation_base(p,n,0.5,beta_gn,1.8,0.5)
    filename = f'{filename}simu{b}.csv'
    testdat.to_csv(filename, index=False)
    phi_data = (np.mean(paramdat['y_11']-paramdat['y_00']))

    # estimate pt1 ps
    ps0outcome = ['t0']
    ps0feature = ['z' + str(x) +'0' for x in range(0,p)]
    ps1outcome = ['t1']
    ps1feature = np.concatenate((['z' + str(x) +'0' for x in range(0,p)],
                                    ['z' + str(x) +'1' for x in range(0,p)],['t0','y0']))
    #Train,Test = train_test_split(testdat,test_size = 0.4,random_state=42)
    

    baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts = ps_function(testdat,ps0feature,ps0outcome,lg=False,l1=True)
    enps0_tr,enps0_ts = ps_predict(testdat,ps0feature,baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts)

    baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts = ps_function(testdat,ps1feature,ps1outcome,lg=False,l1=True)
    enps1_tr,enps1_ts = ps_predict(testdat,ps1feature,baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts)

    # MSM with original ps
    t0 = testdat['t0']; t1= testdat['t1']; y1 = testdat['y1']; y0 = testdat['y0']
    IPW = (t0/((enps0_tr+enps0_ts)/2)+(1-t0)/(1-((enps0_tr+enps0_ts)/2)))*(t1/((enps1_tr+enps1_ts)/2)+(1-t1)/(1-((enps1_tr+enps1_ts)/2)))
    msm_lm = linear_model.LinearRegression()
    msm_lm.fit(testdat[['t0','t1']],testdat['y1'],sample_weight=IPW)
    phi_msm_enl = (np.sum(msm_lm.coef_))
    # SEs
    predictions = msm_lm.predict(testdat[['t0', 't1']])
    residuals = testdat['y1'] - predictions
    # Calculate the variance-covariance matrix
    # X is the matrix of predictors with a column of ones for the intercept
    Xse = np.column_stack((np.ones(len(testdat)), testdat[['t0', 't1']]))
    w_res = residuals**2 * IPW  # Weighted residuals squared
    X_weighted = Xse.T * IPW.ravel()  # Apply weights to the predictors matrix
    # Variance-covariance matrix calculation
    cov_matrix = np.linalg.inv(X_weighted @ Xse) * (X_weighted @ np.diag(w_res) @ Xse).sum()/n
    msmenl_se = np.sqrt(cov_matrix[1,1]+cov_matrix[2,2]+2*cov_matrix[1,2])

    # estimating weights with logistic regression
    lg0 = linear_model.LogisticRegression().fit(testdat[ps0feature],testdat[ps0outcome])
    lmps0 = np.clip(lg0.predict_proba(testdat[ps0feature])[:,1].ravel(),0.01,0.99)
    lg1 = linear_model.LogisticRegression().fit(testdat[ps1feature],testdat[ps1outcome])
    lmps1 = np.clip(lg1.predict_proba(testdat[ps1feature])[:,1].ravel(),0.01,0.99)
    lmP = (t0/lmps0+(1-t0)/(1-lmps0))*(t1/lmps1+(1-t1)/(1-lmps1))
    msm_lg = linear_model.LinearRegression().fit(testdat[['t0','t1']],testdat['y1'],sample_weight=lmP)
    phi_msm_lm = np.sum(msm_lg.coef_)
    # SEs
    lmpredictions = msm_lg.predict(testdat[['t0', 't1']])
    lmresiduals = testdat['y1'] - lmpredictions
    # Calculate the variance-covariance matrix
    lmw_res = lmresiduals**2 * lmP  # Weighted residuals squared
    lmX_weighted = Xse.T * lmP.ravel()  # Apply weights to the predictors matrix
    # Variance-covariance matrix calculation
    lmcov_matrix = np.linalg.inv(lmX_weighted @ Xse) * (lmX_weighted @ np.diag(lmw_res) @ Xse).sum()/n
    msmlm_se = np.sqrt(lmcov_matrix[1,1]+lmcov_matrix[2,2]+2*lmcov_matrix[1,2])

    ## outcome regression ensemble
    iter1feature = ['z{}{}'.format(i, j) for i in range(p) for j in range(2)] + ['y0', 't0', 't1']
    iter1outcome = ['y1']
    iter0feature = ['t0']+['z' + str(x) +'0' for x in range(0,p)]

    preddat11 = testdat.copy(); preddat11['t0']=1;preddat11['t1']=1
    preddat00 = testdat.copy(); preddat00['t0']=0;preddat00['t1']=0
    
    basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts = gcompu_function(testdat,iter1feature,iter1outcome,gbr=True)

    eta2_tr,eta2_ts = gcompu_predict(testdat,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    

     ## G-comp with LR
    lm1 = linear_model.LinearRegression()
    lm1.fit(testdat[iter1feature],testdat[iter1outcome])
    Q11 = lm1.predict(preddat11[iter1feature])
    Q00 = lm1.predict(preddat00[iter1feature])

    lm0_1 = linear_model.LinearRegression()
    lm0_1.fit(testdat[iter0feature],Q11)
    lm0_0 = linear_model.LinearRegression()
    lm0_0.fit(testdat[iter0feature],Q00)
    phi_gcomp_lm = (np.mean(lm0_1.predict(preddat11[iter0feature])-lm0_0.predict(preddat00[iter0feature])))

    ## DR
    
    #eta2 = lm_stack.predict(Q1fit)

    preddat1 = testdat.copy(); preddat1['t1']=1
    preddat0 = testdat.copy(); preddat0['t1']=0

    eta20_tr,eta20_ts = gcompu_predict(preddat0,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
    eta20 = (eta20_tr+eta20_ts)/2
    basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts = gcompu_function(pd.concat([testdat, pd.DataFrame(eta20.reshape(n,), columns=['eta20'])], axis=1),
                                                    iter0feature,'eta20',gbr=True)

    eta21_tr,eta21_ts = gcompu_predict(preddat1,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
    eta21 = (eta21_tr+eta21_ts)/2
    basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts = gcompu_function(pd.concat([testdat, pd.DataFrame(eta21.reshape(n,), columns=['eta21'])], axis=1),
                                                    iter0feature,'eta21',gbr=True)


    Q_00_tr,Q_00_ts= gcompu_predict(preddat00,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    Q_11_tr,Q_11_ts= gcompu_predict(preddat11,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


    # phi_gcomp = (np.mean(Q_11-Q_00))
            
    eta10_tr,eta10_ts = gcompu_predict(preddat0,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    eta11_tr,eta11_ts = gcompu_predict(preddat1,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

    eta0_00_tr = Q_00_tr; eta0_00_ts = Q_00_ts

    preddat10 = testdat.copy(); preddat10['t0']=1;preddat10['t1']=0
    eta0_10_tr,eta0_10_ts = gcompu_predict(preddat10,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    eta0_11_tr = Q_11_tr; eta0_11_ts = Q_11_ts

    preddat01 = testdat.copy(); preddat01['t0']=0;preddat01['t1']=1
    eta0_01_tr,eta0_01_ts = gcompu_predict(preddat01,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


    ## linear equation for DR, subject to change based on MSM/other outcome equation of interest
    # t0 = testdat['t0']; t1= testdat['t1']; y1 = testdat['y1']; y0 = testdat['y0']
    # enps0 = lmps0; enps1 = lmps1
    b0 = [4,2,2];b1 = [2,2,1];b2 = [2,1,2]
    f01_tr = ((t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel()))
    f02_tr = 1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta20_tr.ravel()-eta10_tr.ravel())+1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f03_tr = (eta0_00_tr+eta0_11_tr+eta0_10_tr.ravel()+eta0_01_tr.ravel())
    c0_tr = np.mean(f01_tr+f02_tr+f03_tr)
    

    f11_tr = (t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel())*t0
    f12_tr = t0/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta20_tr.ravel()-eta10_tr.ravel())+t0/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f13_tr = (eta0_11_tr+eta0_10_tr.ravel())
    c1_tr = np.mean(f11_tr+f12_tr+f13_tr)

    f21_tr = (t0/enps0_tr+(1-t0)/(1-enps0_tr))*(t1/enps1_tr+(1-t1)/(1-enps1_tr))*(y1-eta2_tr.ravel())*t1
    f22_tr = 1/((enps0_tr**t0)*(1-enps0_tr)**(1-t0))*(eta21_tr.ravel()-eta11_tr.ravel())
    f23_tr = (eta0_11_tr+eta0_01_tr.ravel())
    c2_tr = np.mean(f21_tr+f22_tr+f23_tr)

    coef_tr = np.linalg.solve(np.array([b0,b1,b2]),np.array([c0_tr,c1_tr,c2_tr]))

    # training set
    f01_ts = ((t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel()))
    f02_ts = 1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta20_ts.ravel()-eta10_ts.ravel())+1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f03_ts = (eta0_00_ts+eta0_11_ts+eta0_10_ts.ravel()+eta0_01_ts.ravel())
    c0_ts = np.mean(f01_ts+f02_ts+f03_ts)

    f11_ts = (t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel())*t0
    f12_ts = t0/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta20_ts.ravel()-eta10_ts.ravel())+t0/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f13_ts = (eta0_11_ts+eta0_10_ts.ravel())
    c1_ts = np.mean(f11_ts+f12_ts+f13_ts)

    f21_ts = (t0/enps0_ts+(1-t0)/(1-enps0_ts))*(t1/enps1_ts+(1-t1)/(1-enps1_ts))*(y1-eta2_ts.ravel())*t1
    f22_ts = 1/((enps0_ts**t0)*(1-enps0_ts)**(1-t0))*(eta21_ts.ravel()-eta11_ts.ravel())
    f23_ts = (eta0_11_ts+eta0_01_ts.ravel())
    c2_ts = np.mean(f21_ts+f22_ts+f23_ts)

    coef_ts = np.linalg.solve(np.array([b0,b1,b2]),np.array([c0_ts,c1_ts,c2_ts]))

    phi_dr = (coef_tr[1]+coef_ts[1])/2+(coef_tr[2]+coef_ts[2])/2

    # empirical sandwitch variance for DR
    Bmat = np.array([[-4,-2,-2],
                     [-2,-2,-1],
                     [-2,-1,-2]])
    Bmatinv = np.linalg.inv(Bmat)
    phi1 = (f01_tr+f01_ts)/2+(f02_tr+f02_ts)/2+(f03_tr+f03_ts)/2-4*(coef_tr[0]+coef_ts[0])/2-2*(coef_tr[1]+coef_ts[1])/2-2*(coef_tr[2]+coef_ts[2])/2
    phi2 = (f11_tr+f11_ts)/2+(f12_tr+f12_ts)/2+(f13_tr+f13_ts)/2-2*(coef_tr[0]+coef_ts[0])/2-2*(coef_tr[1]+coef_ts[1])/2-(coef_tr[2]+coef_ts[2])/2
    phi3 = (f21_tr+f21_ts)/2+(f22_tr+f22_ts)/2+(f23_tr+f23_ts)/2-2*(coef_tr[0]+coef_ts[0])/2-(coef_tr[1]+coef_ts[1])/2-2*(coef_tr[2]+coef_ts[2])/2
    Phi = np.array([phi1,phi2,phi3])
    Fmat = np.zeros((3,3))
    for i in range(n):
        phi_vector = Phi[:,i]
        outer_product = np.outer(phi_vector,phi_vector)
        Fmat += outer_product
    Fmat /= n
    Vmat = (Bmatinv @ Fmat @ Bmatinv.T)/n
    dr_se  = np.sqrt(Vmat[1,1]+Vmat[2,2]+2*Vmat[1,2])

    # SE from Chernozhukov paper
    phi1_dr = (f01_tr)+(f02_tr)+(f03_tr)-4*(coef_tr[0])-2*(coef_tr[1])-2*(coef_tr[2])
    phi2_dr = (f11_tr)+(f12_tr)+(f13_tr)-2*(coef_tr[0])-2*(coef_tr[1])-(coef_tr[2])
    phi3_dr = (f21_tr)+(f22_tr)+(f23_tr)-2*(coef_tr[0])-(coef_tr[1])-2*(coef_tr[2])
    phi1_ds = (f01_ts)+(f02_ts)+(f03_ts)-4*(coef_ts[0])-2*(coef_ts[1])-2*(coef_ts[2])
    phi2_ds = (f11_ts)+(f12_ts)+(f13_ts)-2*(coef_ts[0])-2*(coef_ts[1])-(coef_ts[2])
    phi3_ds = (f21_ts)+(f22_ts)+(f23_ts)-2*(coef_ts[0])-(coef_ts[1])-2*(coef_ts[2])
    Phi = np.column_stack([np.concatenate([phi1_dr,phi1_ds]),np.concatenate([phi2_dr,phi2_ds]),np.concatenate([phi3_dr,phi3_ds])])
    Sigma = np.dot(Phi.T,Phi)/(2*n)
    dr_se_chern = np.sqrt((Sigma[1,1]+Sigma[2,2]+2*Sigma[1,2])/(2*n))

    

    # bootstrap se for Lm Gcomp
    gcomplm_boot = [];msmlm_boot = []
    for i in range(500):
        sampleid = np.random.choice(range(1, n+1), size=n, replace=True)
        dat_boot = testdat.iloc[sampleid-1, :]

        bootdat11 = dat_boot.copy(); bootdat11['t0']=1;bootdat11['t1']=1
        bootdat00 = dat_boot.copy(); bootdat00['t0']=0;bootdat00['t1']=0

        ## G-comp with LR
        lm1boot = linear_model.LinearRegression()
        lm1boot.fit(dat_boot[iter1feature],dat_boot[iter1outcome])
        Q11boot = lm1boot.predict(bootdat11[iter1feature])
        Q00boot = lm1boot.predict(bootdat00[iter1feature])

        lm0_1boot = linear_model.LinearRegression()
        lm0_1boot.fit(dat_boot[iter0feature],Q11boot)
        lm0_0boot = linear_model.LinearRegression()
        lm0_0boot.fit(dat_boot[iter0feature],Q00boot)
        gcomplm_boot.append(np.mean(lm0_1boot.predict(bootdat11[iter0feature])-lm0_0boot.predict(bootdat00[iter0feature])))

        ## MSM with LR
        lgboot0 = linear_model.LogisticRegression().fit(dat_boot[ps0feature],dat_boot[ps0outcome])
        lmpsboot0 = np.clip(lgboot0.predict_proba(dat_boot[ps0feature])[:,1].ravel(),0.01,0.99)
        lgboot1 = linear_model.LogisticRegression().fit(dat_boot[ps1feature],dat_boot[ps1outcome])
        lmpsboot1 = np.clip(lgboot1.predict_proba(dat_boot[ps1feature])[:,1].ravel(),0.01,0.99)
        lmPboot = (dat_boot['t0']/lmpsboot0+(1-dat_boot['t0'])/(1-lmpsboot0))*(dat_boot['t1']/lmpsboot1+(1-dat_boot['t1'])/(1-lmpsboot1))
        msm_lgboot = linear_model.LinearRegression().fit(dat_boot[['t0','t1']],dat_boot['y1'],sample_weight=lmPboot)
        msmlm_boot.append(np.sum(msm_lgboot.coef_))

    gcomplm_se = np.std(gcomplm_boot);msmlmb_se = np.std(msmlm_boot)


    return  phi_data, phi_dr,phi_gcomp_lm, dr_se,gcomplm_se,phi_msm_enl,msmenl_se,phi_msm_lm,msmlm_se,msmlmb_se,dr_se_chern

In [ ]:

from functools import partial
start_time = time.time()
with multiprocess.Pool(processes=20) as pool:
    # phi_data_100bs, phi_gcomp_100bs, phi_dr_100bs, phi_gcomp_lm_100bs, phi_msm_lm_100bs, dr_boot_100bs, dr_btwo_100bs = pool.map(simulation_loop,range(10))
    results1_5 = pool.map(partial(simulation_loop,filename = filename,p = 50),range(10))
    pool.close(); pool.join()

end_time = time.time()
end_time-start_time